In [1]:
%cd ..

/run/media/nazif/2F946E411BA61D49/thesis


In [2]:
import pandas as pd

from scripts.utils_latest import *

# import CLASH and make predictions

In [3]:
clash_df = clash()
# df = find_matches_with_rnaduplex(clash_df)
# df.to_csv("results/0_rnaduplex_predictions_raw.csv", index=False)
df = pd.read_csv("results/0_rnaduplex_predictions_raw.csv")

# data generation & extraction to csv

In [4]:
# adding helper columns from clash
df["id"] = clash_df["id"]
df["clash_mrna_start"] = clash_df["true_start"]
df["clash_mrna_end"] = clash_df["true_end"]
df["true_seed_type"] = clash_df["true_seed_type"]
df["true_num_basepairs"] = clash_df["num_basepairs"]
df["true_seed_basepairs"] = clash_df["seed_basepairs"]
df["true_energy"] = clash_df["folding_energy"]
df["mirna_sequence"] = clash_df["mirna_sequence"]
df["mirna_accession"] = clash_df["mirna_accession"]
df["enst"] = clash_df["enst"]
df["ensg"] = clash_df["ensg"]

In [5]:
# feature generation
df = generate_alignment_string_from_dot_bracket(df)
df = generate_match_count_columns(df)

# adding bp difference columns
df["bp_difference"] = df["pred_num_basepairs"] - df["true_num_basepairs"]
df["seed_bp_difference"] = df["pred_seed_basepairs"] - df["true_seed_basepairs"]

In [6]:
# MRE region

df["mirna_length"] = df["mirna_sequence"].str.len()
df["mre_start"] = df["mrna_start"] - df["mirna_start"] + 1
df["mre_end"] = df["mre_start"] + df["mirna_length"] - 1
df.drop("mirna_length", axis=1, inplace=True)
# mre_start has values less than 1. convert them all into 1s.
df['mre_start'] = df['mre_start'].apply(lambda x: max(x, 1))

# generating mre region sequence
df["full_mrna_seq"] = clash_df["mrna_sequence"]


def slice_column(row):
    return row['full_mrna_seq'][row['mre_start']-1:row['mre_end']]


df['mre_region'] = df.apply(slice_column, axis=1)
# df.drop("full_mrna_seq", axis=1, inplace=True)

In [7]:
# writing to csv
df.to_csv("results/1_generate_data.csv", index=False)
